In [3]:
import numpy as np
import torch
import math 
from torch.utils.data import Dataset


def calculate_fire_spread_rate(wind_speed_at_10m, moisture_content, slope_angle, fuel_load):
    """
    Calculate the rate of spread of a forest fire.

    Parameters:
    wind_speed_at_10m (float): Wind speed at 10 meters height in km/hr.
    moisture_content (float): Moisture content in percentage (35% - 65%).
    slope_angle (float): Slope angle in degrees.
    fuel_load (float): Fuel load in tonnes/hectare (< 6 mm diameter), default is 2 tonnes/hectare.

    Returns:
    float: Rate of spread of the fire in unspecified units.
    """
    # Constants
    a = 1.674
    b = 0.1798
    c = 0.22
    d = 0.158
    e = -0.227
    f = 0.0662

    # Convert wind speed at 10m to wind speed at 1.5m
    wind_speed_at_1_5m = a + b * wind_speed_at_10m

    # Calculate the rate of spread on flat ground
    rate_flat_ground = c * fuel_load * math.exp((wind_speed_at_1_5m ** d) * (moisture_content ** e))

    # Adjust rate for slope
    rate_on_slope = rate_flat_ground * math.exp((slope_angle ** f))

    return rate_on_slope

# Example usage
spread_rate = calculate_fire_spread_rate(20, 50, 10, 2)
spread_rate

2.4078009424508124

In [4]:
from scipy.stats import qmc
import pandas as pd


# Define the bounds for each parameter
bounds = np.array([[11.5, 34.1],  # Wind speed
                   [0, 6],        # Fuel load
                   [35, 65],      # Moisture content
                   [0, 30]])      # Slope angle

# Using Latin Hypercube Sampling (LHS) for a more structured approach
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=40000)
sample = qmc.scale(sample, bounds[:, 0], bounds[:, 1])

# Extracting the samples for each parameter
wind_speed_samples = sample[:, 0]
fuel_samples = sample[:, 1]
moisture_content_samples = sample[:, 2]
slope_angle_samples = sample[:, 3]

# Calculating spread rates for the sampled data
spread_rates = [calculate_fire_spread_rate(wind_speed, moisture, slope, fuel) 
                for wind_speed, moisture, slope, fuel in zip(wind_speed_samples, moisture_content_samples, slope_angle_samples, fuel_samples)]

# Creating a DataFrame to hold the sampled data and calculated spread rates
data_lhs = pd.DataFrame({
    'Wind Speed (km/hr)': wind_speed_samples,
    'Fuel Load (tonnes/hectare)': fuel_samples,
    'Moisture Content (%)': moisture_content_samples,
    'Slope Angle (degrees)': slope_angle_samples,
    'Spread Rate': spread_rates
})

data_lhs.head()

,Wind Speed (km/hr),Fuel Load (tonnes/hectare),Moisture Content (%),Slope Angle (degrees),Spread Rate
0,15.155450,0.857103,56.104848,22.522096,1.069513
1,29.443622,1.984825,46.344758,7.834106,2.426105
2,24.248262,0.091706,46.831901,25.315972,0.121239
3,13.169699,4.352038,45.873803,24.383560,5.557597
4,33.114950,2.835995,46.550878,0.640950,2.932196


from sklearn.preprocessing import MinMaxScaler

# Assuming `data_lhs` is a pandas DataFrame with your data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_lhs.iloc[:, :-1])  # Exclude the target column

# Convert the scaled data back to a DataFrame
scaled_data_lhs = pd.DataFrame(scaled_data, columns=data_lhs.columns[:-1])
scaled_data_lhs['Spread Rate'] = data_lhs['Spread Rate']  # Add the target column back


In [5]:

import torch
from torch.utils.data import Dataset

class FireSpreadDataset(Dataset):
    def __init__(self, dataframe):
        # Convert the dataframe to a PyTorch tensor
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming the last column is the target variable (spread rate)
        features = self.data[idx, :-1]
        target = self.data[idx, -1]
        return features, target

# Convert the DataFrame to a PyTorch Dataset
fire_spread_data = FireSpreadDataset(data_lhs)

# Example: accessing the first item in the dataset
first_sample_features, first_sample_target = fire_spread_data[0]
print(first_sample_features, first_sample_target)


tensor([15.1554,  0.8571, 56.1048, 22.5221]) tensor(1.0695)


In [6]:
print(fire_spread_data[0])

(tensor([15.1554,  0.8571, 56.1048, 22.5221]), tensor(1.0695))


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset
from typing import List 
import torch.nn as nn
import torch.optim as optim

class BushfireModel(nn.Module): 
    def __init__(self, n: int, hidden_layers: List[int]): 
        super().__init__() #Inherit from the nn.module
        self.n = n # n input features
        self.hidden_layers = hidden_layers # hidden layers are the middle ones

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *hidden_layers], [*hidden_layers, 1])
        ]) # Single output

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:    #We don't want to ReLU the last layer
                x = nn.functional.relu(x)  
        return x # Linear output for regression

In [8]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Splitting the dataset
total_size = len(fire_spread_data)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation
train_dataset, val_dataset = random_split(fire_spread_data, [train_size, val_size])

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) #batch size 64 ✔
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Training mode
        for i, (features, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(features)
            loss = loss_criterion(outputs, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # Validation mode
        with torch.no_grad():
            val_loss = 0
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += loss_criterion(outputs, labels.unsqueeze(1)).item()

        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Initialize the model
model = BushfireModel(n=4, hidden_layers=[200,100]) # 2 layers, 1st 200 neurons, 2nd 100 ✔
loss_criterion = nn.MSELoss() #loss fn - Mean Square error ✔
optimizer = optim.Adam(model.parameters(), lr=0.00001) #learning rate 0.00001 ✔

# Train the model
num_epochs = 70
train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs)


Epoch [1/50], Loss: 4.5650, Val Loss: 4.7614
Epoch [2/50], Loss: 3.2916, Val Loss: 4.1762
Epoch [3/50], Loss: 3.7771, Val Loss: 3.5151
Epoch [4/50], Loss: 2.9379, Val Loss: 2.8084
Epoch [5/50], Loss: 1.9078, Val Loss: 2.1169
Epoch [6/50], Loss: 1.5647, Val Loss: 1.4683
Epoch [7/50], Loss: 0.8357, Val Loss: 0.9145
Epoch [8/50], Loss: 0.5207, Val Loss: 0.5010
Epoch [9/50], Loss: 0.2750, Val Loss: 0.2399
Epoch [10/50], Loss: 0.1115, Val Loss: 0.1095
Epoch [11/50], Loss: 0.0569, Val Loss: 0.0574
Epoch [12/50], Loss: 0.0324, Val Loss: 0.0367
Epoch [13/50], Loss: 0.0311, Val Loss: 0.0243
Epoch [14/50], Loss: 0.0226, Val Loss: 0.0175
Epoch [15/50], Loss: 0.0125, Val Loss: 0.0134
Epoch [16/50], Loss: 0.0099, Val Loss: 0.0121
Epoch [17/50], Loss: 0.0088, Val Loss: 0.0089
Epoch [18/50], Loss: 0.0073, Val Loss: 0.0076
Epoch [19/50], Loss: 0.0049, Val Loss: 0.0077
Epoch [20/50], Loss: 0.0067, Val Loss: 0.0057
Epoch [21/50], Loss: 0.0064, Val Loss: 0.0053
Epoch [22/50], Loss: 0.0036, Val Loss: 0.00

In [9]:
def predict_spread_rate(model, wind_speed, moisture_content, fuel_load, slope_angle):
    # Convert input data to tensor
    input_tensor = torch.tensor([wind_speed, moisture_content, fuel_load, slope_angle], dtype=torch.float32)

    # Reshape input_tensor to match the input shape of the model
    input_tensor = input_tensor.unsqueeze(0)  # Adds a batch dimension

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        prediction = model(input_tensor)

    # Return the predicted spread rate
    return prediction.item()

# Example usage
predicted_spread_rate = predict_spread_rate(model, 30, 40, 0, 2)
print("Predicted Spread Rate:", predicted_spread_rate)

# Example usage
spread_rate = calculate_fire_spread_rate(30, 40, 0, 2)
print("Actual Spread Rate:", spread_rate)


Predicted Spread Rate: 20.885269165039062
Actual Spread Rate: 0.7933984307219112


In [10]:
def predict(model, input_features):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(input_features)
    return predictions

# Example usage with one sample from the validation set
features, labels = next(iter(val_loader))
prediction = predict(model, features)
print(f'Predicted: {prediction}, Actual: {labels}')


Predicted: tensor([[6.2714],
        [2.5037],
        [6.5373],
        [1.0233],
        [0.4574],
        [0.8579],
        [4.6741],
        [7.1812],
        [1.1075],
        [6.5196],
        [3.9453],
        [2.6720],
        [4.4857],
        [6.4140],
        [4.3520],
        [4.4433],
        [4.6117],
        [4.7205],
        [1.4855],
        [1.6478],
        [5.0301],
        [1.3669],
        [0.1703],
        [2.7069],
        [1.4894],
        [2.6519],
        [2.6115],
        [7.4724],
        [0.1117],
        [2.1688],
        [3.9662],
        [1.8231],
        [0.8442],
        [3.6517],
        [5.5991],
        [6.4136],
        [2.8897],
        [7.5480],
        [5.7851],
        [0.1037],
        [0.4421],
        [4.9349],
        [4.0006],
        [2.5811],
        [0.1868],
        [1.7129],
        [4.2262],
        [3.4581],
        [1.9155],
        [6.9036],
        [4.4591],
        [1.5695],
        [3.4569],
        [2.2567],
        [5.0839],